# ⚡ Notebook Otimizado: Geração Rápida com Retroalimentação

**Estratégias de Performance:**
- ✅ Menos passos de inferência (10-15 vs 20)
- ✅ Dimensões menores (256x256 vs 512x512) 
- ✅ Menos frames (6-8 vs 12)
- ✅ Strength otimizado (0.5-0.6)
- ✅ Scheduler mais rápido
- ✅ Pipeline otimizado

**Tempo estimado:** 2-4 minutos total (vs 12-24 minutos)


In [1]:
# Instalação rápida
%pip -q install diffusers==0.30.0 transformers accelerate safetensors "imageio[ffmpeg]" pillow



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ⚡ VERSÃO ULTRA-RÁPIDA: 2-4 minutos total
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionImg2ImgPipeline, EulerDiscreteScheduler
import time

print("⚡ Iniciando geração ULTRA-RÁPIDA...")
start_time = time.time()

# Configuração OTIMIZADA para velocidade
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 256, 256  # ← MENOR = MAIS RÁPIDO
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 10, 7.0, 6, 12  # ← MENOS STEPS + FRAMES
strength = 0.5  # ← MAIS SUAVE = MAIS RÁPIDO
negative_prompt = "blurry, low quality"

print(f"📊 Config: {width}x{height}, {steps} steps, {frames} frames")

# Carregar imagem
img_path = "/Users/alansms/stable_diffusion_video_generator/homer.png"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)
print(f"✅ Imagem carregada: {init_img.size}")

# Pipeline OTIMIZADO
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Dispositivo: {device}")

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)

# Scheduler MAIS RÁPIDO
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

# Otimizações adicionais
if device == "cuda":
    pipe.enable_attention_slicing()  # Reduz VRAM e acelera
    pipe.enable_memory_efficient_attention()  # Otimização de memória

print("✅ Pipeline otimizado pronto")

# Geração RÁPIDA
base_prompt = "cinematic, detailed, high quality, maintain original subject"
frames_list = [init_img]
current = init_img

print(f"🎞️ Gerando {frames} frames (modo rápido)...")
for i in range(1, frames):
    frame_start = time.time()
    print(f"  Frame {i+1}/{frames}...", end=" ")
    
    out = pipe(
        prompt=base_prompt,
        negative_prompt=negative_prompt,
        image=current,
        strength=strength,
        num_inference_steps=steps,
        guidance_scale=guidance,
    )
    
    img = out.images[0]
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    
    frames_list.append(img)
    current = img
    
    frame_time = time.time() - frame_start
    print(f"✅ ({frame_time:.1f}s)")

# Exportar
os.makedirs("output", exist_ok=True)
out_path = "output/video_ultra_rapido.mp4"
with imageio.get_writer(out_path, fps=fps, quality=6) as w:  # quality=6 = mais rápido
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))

total_time = time.time() - start_time
print(f"🎉 CONCLUÍDO em {total_time:.1f} segundos!")
print(f"✅ Vídeo salvo: {out_path}")
print(f"📊 {len(frames_list)} frames, {fps} FPS, {width}x{height}px")
print(f"⚡ Tempo médio por frame: {total_time/frames:.1f}s")


⚡ Iniciando geração ULTRA-RÁPIDA...
📊 Config: 256x256, 10 steps, 6 frames
✅ Imagem carregada: (256, 256)
🔧 Dispositivo: cpu


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 35.64it/s]


✅ Pipeline otimizado pronto
🎞️ Gerando 6 frames (modo rápido)...
  Frame 2/6... 

100%|██████████| 5/5 [00:07<00:00,  1.55s/it]


✅ (10.7s)
  Frame 3/6... 

...

✅ (7.0s)
  Frame 5/6... 

100%|██████████| 5/5 [00:05<00:00,  1.00s/it]


✅ (6.9s)
  Frame 6/6... 

100%|██████████| 5/5 [00:05<00:00,  1.16s/it]


✅ (7.7s)
🎉 CONCLUÍDO em 41.2 segundos!
✅ Vídeo salvo: output/video_ultra_rapido.mp4
📊 6 frames, 12 FPS, 256x256px
⚡ Tempo médio por frame: 6.9s


In [4]:
# 🚀 VERSÃO BALANCEADA: Qualidade vs Velocidade (4-6 minutos)
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler
import time

print("🚀 Iniciando geração BALANCEADA...")
start_time = time.time()

# Configuração BALANCEADA
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 384, 384  # ← Tamanho médio
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 15, 7.5, 8, 12  # ← Balanceado
strength = 0.6
negative_prompt = "blurry, low quality, distorted"

print(f"📊 Config: {width}x{height}, {steps} steps, {frames} frames")

# Carregar imagem
img_path = "/Users/alansms/stable_diffusion_video_generator/modelo-1.png"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)
print(f"✅ Imagem carregada: {init_img.size}")

# Pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

if device == "cuda":
    pipe.enable_attention_slicing()

print("✅ Pipeline balanceado pronto")

# Geração
base_prompt = "cinematic, detailed, high quality, maintain original subject"
frames_list = [init_img]
current = init_img

print(f"🎞️ Gerando {frames} frames (modo balanceado)...")
for i in range(1, frames):
    frame_start = time.time()
    print(f"  Frame {i+1}/{frames}...", end=" ")
    
    out = pipe(
        prompt=base_prompt,
        negative_prompt=negative_prompt,
        image=current,
        strength=strength,
        num_inference_steps=steps,
        guidance_scale=guidance,
    )
    
    img = out.images[0]
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    
    frames_list.append(img)
    current = img
    
    frame_time = time.time() - frame_start
    print(f"✅ ({frame_time:.1f}s)")

# Exportar
os.makedirs("output", exist_ok=True)
out_path = "output/video_balanceado.mp4"
with imageio.get_writer(out_path, fps=fps, quality=8) as w:
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))

total_time = time.time() - start_time
print(f"🎉 CONCLUÍDO em {total_time:.1f} segundos!")
print(f"✅ Vídeo salvo: {out_path}")
print(f"📊 {len(frames_list)} frames, {fps} FPS, {width}x{height}px")
print(f"⚡ Tempo médio por frame: {total_time/frames:.1f}s")


🚀 Iniciando geração BALANCEADA...
📊 Config: 384x384, 15 steps, 8 frames
✅ Imagem carregada: (384, 384)


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 40.79it/s]


✅ Pipeline balanceado pronto
🎞️ Gerando 8 frames (modo balanceado)...
  Frame 2/8... 

100%|██████████| 9/9 [00:13<00:00,  1.55s/it]


✅ (17.5s)
  Frame 3/8... 

...

✅ (17.4s)
  Frame 7/8... 

100%|██████████| 9/9 [00:13<00:00,  1.49s/it]


✅ (16.8s)
  Frame 8/8... 

100%|██████████| 9/9 [00:13<00:00,  1.45s/it]


✅ (16.5s)
🎉 CONCLUÍDO em 123.3 segundos!
✅ Vídeo salvo: output/video_balanceado.mp4
📊 8 frames, 12 FPS, 384x384px
⚡ Tempo médio por frame: 15.4s
